In [6]:
import pygame
import numpy as np
from typing import List, Tuple
from enum import Enum

# Initialize Pygame
pygame.init()

# Constants
WIDTH = 800
HEIGHT = 600
FPS = 60
G = 6.67430e-11  # gravitational constant
SCALE = 1e9  # scale factor to convert meters to pixels
TIME_STEP = 3600  # simulation time step in seconds (1 hour)
VELOCITY_SCALE = 1e4  # scale factor for launch velocity

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
GRAY = (128, 128, 128)

class BodySize(Enum):
    SMALL = {"mass": 1e23, "radius": 5}
    MEDIUM = {"mass": 1e24, "radius": 8}
    LARGE = {"mass": 1e25, "radius": 12}

class Body:
    def __init__(self, mass: float, position: np.ndarray, velocity: np.ndarray, color: Tuple[int, int, int], radius: int):
        self.mass = mass
        self.position = position  # in meters
        self.velocity = velocity  # in m/s
        self.color = color
        self.radius = radius  # display radius in pixels
        
    def update_position(self, acceleration: np.ndarray, time_step: float):
        self.velocity += acceleration * time_step
        self.position += self.velocity * time_step
        
    def draw(self, screen):
        # Convert position from meters to pixels and center in screen
        pixel_pos = (self.position / SCALE + np.array([WIDTH/2, HEIGHT/2])).astype(int)
        pygame.draw.circle(screen, self.color, pixel_pos, self.radius)

class NBodySimulation:
    def __init__(self):
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("N-Body Simulation")
        self.clock = pygame.time.Clock()
        self.bodies = self.initialize_bodies()
        self.dragging = False
        self.drag_start = None
        self.current_size = BodySize.MEDIUM
        self.paused = False
        self.font = pygame.font.Font(None, 36)
        
    def initialize_bodies(self) -> List[Body]:
        # Create some example bodies (Sun and planets)
        bodies = [
            # Sun (at center)
            Body(
                mass=1.989e30,  # kg
                position=np.array([0.0, 0.0]),  # m
                velocity=np.array([0.0, 0.0]),  # m/s
                color=YELLOW,
                radius=20
            ),
            # Earth
            Body(
                mass=5.972e24,  # kg
                position=np.array([149.6e9, 0.0]),  # m (1 AU)
                velocity=np.array([0.0, 29.78e3]),  # m/s
                color=BLUE,
                radius=10
            )
        ]
        return bodies
    
    def calculate_acceleration(self, body: Body, other_bodies: List[Body]) -> np.ndarray:
        total_acceleration = np.zeros(2)
        for other in other_bodies:
            if other is not body:
                r = other.position - body.position
                r_mag = np.linalg.norm(r)
                if r_mag > 0:  # Avoid division by zero
                    total_acceleration += G * other.mass * r / (r_mag ** 3)
        return total_acceleration
    
    def screen_to_space_coords(self, screen_pos: Tuple[int, int]) -> np.ndarray:
        return (np.array(screen_pos) - np.array([WIDTH/2, HEIGHT/2])) * SCALE
    
    def handle_click_drag(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            self.dragging = True
            self.drag_start = np.array(event.pos)
        
        elif event.type == pygame.MOUSEBUTTONUP and event.button == 1 and self.dragging:
            self.dragging = False
            end_pos = np.array(event.pos)
            
            # Calculate velocity based on drag vector
            launch_vector = self.drag_start - end_pos
            velocity = launch_vector * VELOCITY_SCALE
            
            # Create new body
            new_body = Body(
                mass=self.current_size.value["mass"],
                position=self.screen_to_space_coords(self.drag_start),
                velocity=velocity,
                color=GREEN,
                radius=self.current_size.value["radius"]
            )
            self.bodies.append(new_body)
    
    def handle_input(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_1]:
            self.current_size = BodySize.SMALL
        elif keys[pygame.K_2]:
            self.current_size = BodySize.MEDIUM
        elif keys[pygame.K_3]:
            self.current_size = BodySize.LARGE
        elif keys[pygame.K_SPACE]:
            self.paused = not self.paused
    
    def draw_ui(self):
        # Draw size indicator
        size_text = f"Size: {self.current_size.name}"
        text_surface = self.font.render(size_text, True, WHITE)
        self.screen.blit(text_surface, (10, 10))
        
        # Draw pause indicator if paused
        if self.paused:
            pause_text = "PAUSED"
            text_surface = self.font.render(pause_text, True, WHITE)
            self.screen.blit(text_surface, (WIDTH - 100, 10))
        
        # Draw drag line when dragging
        if self.dragging:
            mouse_pos = pygame.mouse.get_pos()
            pygame.draw.line(self.screen, WHITE, self.drag_start, mouse_pos, 2)
    
    def update(self):
        if not self.paused:
            # Calculate and apply accelerations for all bodies
            accelerations = []
            for body in self.bodies:
                acc = self.calculate_acceleration(body, self.bodies)
                accelerations.append(acc)
                
            # Update positions
            for body, acc in zip(self.bodies, accelerations):
                body.update_position(acc, TIME_STEP)
    
    def draw(self):
        self.screen.fill(BLACK)
        for body in self.bodies:
            body.draw(self.screen)
        self.draw_ui()
        pygame.display.flip()
    
    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        running = False
                self.handle_click_drag(event)
            
            self.handle_input()
            self.update()
            self.draw()
            self.clock.tick(FPS)
        
        pygame.quit()

if __name__ == "__main__":
    simulation = NBodySimulation()
    simulation.run()

In [2]:
import numpy as np
import pygame
import sys

def n_body_deriv(t, y, p):
    """
    Computes the time derivative dy/dt for an n-body system.

    Parameters:
      t : float
          Time (unused, but required for standard ODE signatures).
      y : numpy.ndarray
          State vector, flattened as [x0, y0, x1, y1, ..., vx0, vy0, vx1, vy1, ...].
          The first half contains positions and the second half velocities.
      p : dict
          Dictionary of parameters. Required keys:
            - "G": Gravitational constant.
            - "masses": 1D numpy array of masses for each body.
            - "dim": Dimension of the simulation (e.g. 2 for a 2D simulation).
            - "fix_first": Boolean; if True the first body's acceleration (and velocity) is zero.
            - "softening": (Optional) A small number added to the distance to avoid singularities.
    
    Returns:
      dydt : numpy.ndarray
          Derivative of the state vector, with the same shape as y.
    """
    G = p["G"]
    masses = p["masses"]
    n = masses.shape[0]
    d = p["dim"]
    softening = p.get("softening", 1e-3)
    
    # Separate positions and velocities from the state vector.
    positions = y[:n*d].reshape((n, d))
    velocities = y[n*d:].reshape((n, d))
    
    # Initialize accelerations to zero.
    accelerations = np.zeros_like(positions)
    
    # Compute pairwise gravitational accelerations without double counting.
    for i in range(n):
        for j in range(i+1, n):
            # Vector from body j to body i.
            r_vec = positions[i] - positions[j]
            # Compute distance with a softening factor to prevent singularities.
            r = np.linalg.norm(r_vec) + softening
            # Gravitational force leads to acceleration:
            # a_i = -G * m_j * (r_i - r_j) / r^3  and  a_j = G * m_i * (r_i - r_j) / r^3
            factor = G / (r**3)
            a_i = -factor * masses[j] * r_vec
            a_j =  factor * masses[i] * r_vec
            
            accelerations[i] += a_i
            accelerations[j] += a_j

    # If fix_first is set, the first body does not move.
    if p.get("fix_first", False):
        accelerations[0] = 0.0
        velocities[0] = 0.0  # Ensure its velocity remains zero.
    
    # The derivative of positions is velocities; the derivative of velocities is accelerations.
    dydt = np.concatenate((velocities.flatten(), accelerations.flatten()))
    return dydt

def euler_step(func, t, y, dt, p):
    """
    Advances the state y by one time step dt using Euler's method.
    """
    return y + dt * func(t, y, p)

def main():
    # Initialize Pygame.
    pygame.init()
    width, height = 800, 600
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption("N-Body Planetary Simulation")
    clock = pygame.time.Clock()
    
    # Simulation parameters.
    dim = 2                  # 2D simulation.
    G = 1.0                  # Gravitational constant.
    dt = 0.01                # Time step.
    
    # Define the bodies:
    # In this example, we have three bodies:
    #   - Body 0: a heavy, fixed "sun" at the center.
    #   - Body 1 and 2: two lighter "planets" with initial positions and velocities.
    n = 3
    masses = np.array([1000.0, 1.0, 1.0])
    
    # Set initial positions (in pixel coordinates).
    positions = np.array([
        [width / 2, height / 2],      # Sun at center.
        [width / 2 + 100, height / 2],  # Planet 1.
        [width / 2 - 150, height / 2]   # Planet 2.
    ], dtype=float)
    
    # Set initial velocities.
    # Here we choose velocities that will roughly produce orbital motion.
    velocities = np.array([
        [0, 0],       # Sun (fixed if fix_first is True).
        [0, 2.5],     # Planet 1.
        [0, -2.0]     # Planet 2.
    ], dtype=float)
    
    # Flatten the state vector: first all positions then all velocities.
    y = np.concatenate((positions.flatten(), velocities.flatten()))
    
    # Define the parameter dictionary.
    p = {
        "G": G,
        "masses": masses,
        "dim": dim,
        "fix_first": True,   # Fix the sun.
        "softening": 1e-2
    }
    
    # Main simulation loop.
    running = True
    t = 0.0
    while running:
        # Event handling.
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        # Update the state using Euler integration.
        y = euler_step(n_body_deriv, t, y, dt, p)
        t += dt
        
        # Clear the screen.
        screen.fill((0, 0, 0))
        
        # Extract and reshape positions from the state vector.
        positions = y[:n*dim].reshape((n, dim))
        
        # Draw the bodies.
        for i in range(n):
            # Choose color and size based on the body.
            if i == 0:
                color = (255, 255, 0)  # Sun: yellow.
                radius = 10
            else:
                color = (0, 255, 255)  # Planets: cyan.
                radius = 5
            # Convert position coordinates to integers.
            pos_int = (int(positions[i, 0]), int(positions[i, 1]))
            pygame.draw.circle(screen, color, pos_int, radius)
        
        pygame.display.flip()
        clock.tick(240)  # Limit to 60 FPS.
    
    pygame.quit()
    sys.exit()

if __name__ == '__main__':
    main()


SystemExit: 